In [1]:
# Imports

import numpy as np
from sklearn.metrics import f1_score, average_precision_score
import math
import requests
import os

import datetime
from datetimerange import DateTimeRange
from datetime import timedelta  
from dateutil import parser
from urllib import *
import json

from sklearn.cluster import KMeans

from pathlib import Path
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import string

from collections import Counter
from tqdm import tqdm_notebook 
tqdm_notebook().pandas()
from tqdm import tqdm_notebook as tqdm

import requests
import urllib

import re
import glob
import time 

import itertools
import pathlib

from typing import Union
import pickle

import glob


# Greedy IDE completion 
%config IPCompleter.greedy=True

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; } </style>"))
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 60)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [2]:
os.chdir('/home/winston/PycharmProjects/LegalInteligenceTFIDF/')
print(os.getcwd())
df = pd.read_hdf(path_or_buf='./data/df_27_aggregated')


/home/winston/PycharmProjects/LegalInteligenceTFIDF


In [5]:
df

,UserID,DocumentIDs,DocumentPositions,DocumentURLs,EventID,ID,SearchTypeID,ShortTimeStamp,SearchText,SearchText_norm,idf_query_score,query_length,score_based_2_split,score_based_3_split,score_based_5_split,score_based_7_split,score_based_10_split,score_based_15_split,score_based_18_split,even_size_bins_2,even_size_bins_3,even_size_bins_5,even_size_bins_7,even_size_bins_10,even_size_bins_15
7193,11028,"[32295235, 31998893]","[1, 11]",[https://www.navigator.nl/#/document/id4aee20c...,27,a9e0d162-42f1-4785-9d8c-3daef31915da,14,20190809,NJ,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
20091,24639,32246565,[6],https://www.navigator.nl/#/document/id292e754a...,27,ad64390a-887a-4c43-82e3-30be17118109,14,20190722,nj,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
19971,24639,32133620,[12],https://www.navigator.nl/#/document/id2462c20d...,27,508e1b9b-2bf9-4dd3-aa0a-a989a841d9b9,14,20190702,nj,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
288240,201539,32159295,[1],https://www.navigator.nl/#/document/id97153aa3...,27,507044b9-1efc-481c-a0c4-82a4b6e1dbf2,14,20190628,NJ,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
288242,201539,32159295,[1],https://www.navigator.nl/#/document/id97153aa3...,27,a136c29e-2387-4acc-beba-6b40a6189c9e,14,20190701,NJ,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
20074,24639,"[32246569, 32246565, 32246565]","[4, 6, 6]",[https://www.navigator.nl/#/document/idf06cd19...,27,438cdbe8-2252-4e8f-bace-4bb53c7b843f,14,20190718,nj,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
295418,206183,32042319,[1],https://www.navigator.nl/#/document/id68265838...,27,eaa9e23d-2bee-4907-b70e-feac925bf7f8,14,20190601,NJ,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
397532,233695,31737661,[7],https://www.navigator.nl/#/document/ida06a6720...,27,a6526ebd-7fce-4a62-9250-9e7974f55098,14,20190821,NJ,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10270,13426,"[32084878, 32084878]","[1, 1]",[https://www.navigator.nl/#/document/ide0621f9...,27,3212d70f-22d0-4ef1-b3e4-56db56aee911,14,20190613,nj,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0
356191,226258,"[32119208, 32229897]","[50, 7]",[https://www.navigator.nl/#/document/idd588353...,27,63e02115-75f6-47eb-9ee9-3e684dfe9870,14,20190717,nj,[nj],3.766668,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
from collections import Counter

search_texts = df['SearchText_norm'].tolist()
search_texts = [x for y in search_texts for x in y if not x.isdigit()]

Counter(search_texts)

Counter({'nj': 28342,
         'artikel': 11577,
         'recht': 3124,
         'jor': 8293,
         'hr': 7878,
         'bw': 13276,
         'rv': 4112,
         'strafrecht': 975,
         'art': 8805,
         'jar': 4822,
         'verjaring': 1686,
         'wet': 4692,
         'lid': 6381,
         'ondernemingsraad': 686,
         'auteursrecht': 643,
         'een': 4159,
         'faillissement': 1882,
         'fw': 1207,
         '843a': 773,
         'evrm': 955,
         'avg': 1380,
         'maart': 1651,
         'ehrm': 748,
         'hoge': 2254,
         'raad': 2947,
         'commentaar': 1020,
         'beroep': 1631,
         'ondernemingsrecht': 1573,
         'awb': 1997,
         'mei': 1318,
         'december': 1556,
         'eurocommerce': 50,
         'bestuurder': 1684,
         'f': 166,
         'bibob': 314,
         'wob': 788,
         'februari': 1594,
         'oktober': 1418,
         'ab': 3237,
         'boek': 507,
         'vof': 363,
 

In [28]:
df_common = pd.DataFrame({'SearchText' : search_texts})
df

,UserID,DocumentIDs,DocumentPositions,DocumentURLs,EventID,ID,SearchTypeID,ShortTimeStamp,SearchText,SearchText_norm,idf_query_score,score_based_15_split,score_based_10_split,score_based_5_split,score_based_3_split,score_based_2_split,even_size_bins_15,even_size_bins_10,even_size_bins_5,even_size_bins_3,even_size_bins_2
7193,11028,"[32295235, 31998893]","[1, 11]",[https://www.navigator.nl/#/document/id4aee20c...,27,a9e0d162-42f1-4785-9d8c-3daef31915da,14,20190809,NJ,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
330158,219507,32119204,[1],https://www.navigator.nl/#/document/id93749ed4...,27,6ce4d6c9-ef17-4aa9-9029-013ee66b10ed,14,20190620,nj,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
19040,23955,"[32393522, 3794163]","[1, 3]",[https://www.navigator.nl/#/document/id47d2565...,27,273cae6d-b0fd-4551-b922-724e16cb866c,14,20190821,nj,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
262712,189733,14980884,[4],https://www.navigator.nl/#/document/id72c033ff...,27,6f3cb0f2-fc85-472e-a435-ee6e2b78601d,14,20190614,nj,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
288258,201539,32295235,[1],https://www.navigator.nl/#/document/id4aee20cc...,27,a9526c06-228a-4c1d-a078-6f57afbd0320,14,20190723,NJ,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
288259,201539,14923852,[11],https://www.navigator.nl/#/document/id7fd738f7...,27,cf001e7b-0665-4308-9915-168b8f77674f,14,20190724,NJ,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
832,932,"[32042320, 32042315, 32042324, 32042316]","[5, 7, 6, 8]",[https://www.navigator.nl/#/document/id71df9f9...,27,42e7b23b-f94b-45e0-be1b-0359554c5de6,14,20190603,nj,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
25773,32964,32133617,[1],https://www.navigator.nl/#/document/id054d354d...,27,c93a1209-8e6c-45ae-b232-a3cd678a9d9d,14,20190621,nj,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
19921,24639,"[32084883, 32084859]","[6, 5]",[https://www.navigator.nl/#/document/idfe9bb28...,27,1b901df4-5987-40bd-a939-4341ae4dd91a,14,20190612,nj,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0
272132,192908,"[31969297, 31852479, 32101963]","[3, 12, 38]",[https://www.navigator.nl/#/document/idfeab139...,27,dd2c23cc-b17a-4353-909b-97bd5fc74f33,14,20190621,NJ,[nj],3.766668,bucket_0,bucket_0,bucket_0,bucket_0,bucket_0,0,0,0,0,0


In [30]:
most = df_common['SearchText'].value_counts()
most

nj                   28342
bw                   13276
artikel              11577
art                   8805
jor                   8293
                     ...  
geleidehond              1
renovatiesubsidie        1
gelast”                  1
dickman                  1
deeltijdww               1
Name: SearchText, Length: 96227, dtype: int64

In [ ]:

df_common.groupby(['SearchText']).size().sort_values(ascending=False).transform(lambda x: x/x.sum()).reset_index(name='count')
    

In [35]:
print(os.listdir('./data/'))
stopwords = np.loadtxt('./data/stopwords_nl.txt', dtype=str)

['idf_dict.pkl', 'click_through_rate_rank15.jpeg', 'df_27', 'stopwords_nl.txt', 'click_through_rate_rank50.jpeg', 'data.txt', '.~lock.search_text_6_10.xlsx#', 'solr_dump.txt', 'click_through_rate_rank10.jpeg', 'search_text_6_10.xlsx', 'df_agg', 'titles.txt', 'click_through_rate_rank5.jpeg', 'df_27_aggregated']


In [36]:
stopwords

array(['aan', 'aangaande', 'aangezien', 'achter', 'achterna', 'afgelopen',
       'al', 'aldaar', 'aldus', 'alhoewel', 'alias', 'alle', 'allebei',
       'alleen', 'alsnog', 'altijd', 'altoos', 'ander', 'andere',
       'anders', 'anderszins', 'behalve', 'behoudens', 'beide', 'beiden',
       'ben', 'beneden', 'bent', 'bepaald', 'betreffende', 'bij',
       'binnen', 'binnenin', 'boven', 'bovenal', 'bovendien',
       'bovengenoemd', 'bovenstaand', 'bovenvermeld', 'buiten', 'daar',
       'daarheen', 'daarin', 'daarna', 'daarnet', 'daarom', 'daarop',
       'daarvanlangs', 'dan', 'dat', 'de', 'die', 'dikwijls', 'dit',
       'door', 'doorgaand', 'dus', 'echter', 'eer', 'eerdat', 'eerder',
       'eerlang', 'eerst', 'elk', 'elke', 'en', 'enig', 'enigszins',
       'enkel', 'er', 'erdoor', 'even', 'eveneens', 'evenwel', 'gauw',
       'gedurende', 'geen', 'gehad', 'gekund', 'geleden', 'gelijk',
       'gemoeten', 'gemogen', 'geweest', 'gewoon', 'gewoonweg', 'haar',
       'had', 'hadden'

In [10]:
dirs = os.listdir('./data/AOL-user-ct-collection')

In [18]:
text_files_aol = glob.glob('./data/AOL-user-ct-collection/*.txt')

text_1 = pd.read_csv(text_files_aol[0],  sep='')  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
text_1

,AnonID\tQuery\tQueryTime\tItemRank\tClickURL
0,36\thelena ga\t2006-03-01 10:30:46
1,36\thelena ga\t2006-03-01 10:31:34
2,36\tcheceron s\t2006-03-01 10:45:23
3,36\tcheveron gas station\t2006-03-01 10:46:07
4,36\tcheveron gas station richmond virginia\t20...
5,36\tcheveron glenside road richmond virginia\t...
6,36\tchevron glenside road richmond virginia\t2...
7,36\ttrain\t2006-03-04 10:08:21\t1\thttp://www....
8,36\tpeking richmond virginia\t2006-03-04 11:01...
9,36\tpeking richmond virginia\t2006-03-04 11:01...
